# Local binary pattern - on artificial data
Came from 2020-09-04_Local_binary_pattern_with_covid
and came from 2020-09-29_Local_binary_pattern_with_covid
and came from 2020-10-11_Local_binary_pattern_with_covid

In [22]:
import matplotlib.pyplot as plt
import numpy as np
import math
from PIL import Image
import cv2
import skimage
import tifffile
import os
import pandas as pd

import platform
if platform.system() == 'Windows':
    fileroot = 'X:/'
    print('Loading Windows')
if platform.system() == 'Linux':
    fileroot = '/home/jovyan/'
    print('Loading Linux')
if False:
    fileroot = '/home/ubuntu/Documents/'

colors_new=['#0072b2','#009e73','#d55e00', '#cc79a7','#f0e442','#56b4e9']

import sys
sys.path.append(fileroot+'VU_TEX/Ben_utilities/')
import useful_functions as uf 
import time

Loading Linux


In [23]:
import matplotlib
import matplotlib.patheffects as path_effects
from functools import partial
from skimage.feature import local_binary_pattern
from importlib import reload
from itertools import repeat
from datetime import datetime
from tqdm.notebook import trange, tqdm
from joblib import Parallel, delayed
from numba import njit

In [24]:
reload(uf)

<module 'useful_functions' from '/home/jovyan/VU_TEX/Ben_utilities/useful_functions.py'>

In [25]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [26]:
directory_out_top = fileroot + 'VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/'

In [27]:
directory_in_imgs = fileroot + 'VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-06d_image_conversion_and_crop_to_small_version/'
#imgs = os.listdir(directory_in_imgs)
#if 'Thumbs.db' in imgs:
#    imgs.remove('Thumbs.db')
#imgs.sort(key=partial(uf.getvaluefromstringbest, variable='MS', ending='.', mydtype=int))
imgs = ['V11J11-099__Z4_FO3_01__A1_small.tif']
#imgs = [img for img in imgs if '.npy' in img]
imgs

['V11J11-099__Z4_FO3_01__A1_small.tif']

In [28]:
radius_list = [round(1.499*1.327**(float(x))) for x in range(0, 15)]
print(radius_list); len(radius_list)

[1, 2, 3, 4, 5, 6, 8, 11, 14, 19, 25, 34, 45, 59, 79]


15

In [29]:
channel_list = [0, 1, 2]
print(channel_list)

[0, 1, 2]


In [30]:
#directory_metadata = fileroot + 'VU_TEX/playground_Ben/2022-02-17_UK_CIC/2022-09-06_Iridium_paper_analysis/2022-09-09_merge_metadata_into_iridium_metadata/'
#filename_metadata = '2022-09-12_IMC_and_HnE.xlsx'
#df2 = pd.read_excel(directory_metadata + filename_metadata, engine='openpyxl')
#df2

In [31]:
#df_all = pd.DataFrame(imgs, columns=['Filenames'])
#df_all

In [32]:
#result = df_all.merge(df2, how='outer', indicator=True)
#result[result['_merge'] != 'both']

In [33]:
def get_dims_tiff(filepath):
    myimg = skimage.io.imread(filepath)
    return myimg.shape

df_all = pd.DataFrame(imgs, columns=['Filenames'])
#df_all['dims'] = list(map(uf.get_dims_from_numpy_file, [directory_in_imgs]*len(df_all) + df_all['Filenames']))
df_all['dims'] = list(map(get_dims_tiff, [directory_in_imgs]*len(df_all) + df_all['Filenames']))
#df_all['dims_superpixel'] = list(map(uf.get_cropped_superpixel_dims, df_all['dims'], repeat(1)))

df_all = uf.create_df_cross_for_separate_dfs(df_all,
                                             pd.DataFrame(radius_list, columns=['radius']))

df_all = uf.create_df_cross_for_separate_dfs(df_all,
                                             pd.DataFrame(channel_list , columns=['channel']))

#df_all['n_points'] = [int(32/5*r) for r in df_all['radius']]
df_all['n_points'] = [math.ceil(2*math.pi*r) for r in df_all['radius']]
df_all['method'] = ['uniform']*len(df_all)

df_all['Subfolder'] = ['Ch_' + str(channel) + '_METHOD_'+ method + '_npoints_' + str(n_points) + '_radius_' + str(radius) + '/' for
                      channel, method, n_points, radius in zip(df_all['channel'],
                                                      df_all['method'], 
                                                          df_all['n_points'],
                                                         df_all['radius'])]

df_all['Fpath_out'] = [directory_out_top + subfolder_out + filename.replace('.tif', '.npy')
                      for subfolder_out, filename
                       in zip(df_all['Subfolder'], df_all['Filenames'])]

df_all['patchsize'] = 100

df_all.sort_values(['n_points', 'Filenames'], inplace=True,
                  ascending=True)
df_all.reset_index(drop=True, inplace=True)

df_all

,Filenames,dims,radius,channel,n_points,method,Subfolder,Fpath_out,patchsize
0,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,0,7,uniform,Ch_0_METHOD_uniform_npoints_7_radius_1/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy,100
1,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,1,7,uniform,Ch_1_METHOD_uniform_npoints_7_radius_1/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_1_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy,100
2,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,2,7,uniform,Ch_2_METHOD_uniform_npoints_7_radius_1/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_2_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy,100
3,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,0,13,uniform,Ch_0_METHOD_uniform_npoints_13_radius_2/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy,100
4,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,1,13,uniform,Ch_1_METHOD_uniform_npoints_13_radius_2/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_1_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy,100
5,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,2,13,uniform,Ch_2_METHOD_uniform_npoints_13_radius_2/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_2_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy,100
6,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,0,19,uniform,Ch_0_METHOD_uniform_npoints_19_radius_3/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_19_radius_3/V11J11-099__Z4_FO3_01__A1_small.npy,100
7,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,1,19,uniform,Ch_1_METHOD_uniform_npoints_19_radius_3/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_1_METHOD_uniform_npoints_19_radius_3/V11J11-099__Z4_FO3_01__A1_small.npy,100
8,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,2,19,uniform,Ch_2_METHOD_uniform_npoints_19_radius_3/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_2_METHOD_uniform_npoints_19_radius_3/V11J11-099__Z4_FO3_01__A1_small.npy,100
9,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",4,0,26,uniform,Ch_0_METHOD_uniform_npoints_26_radius_4/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_26_radius_4/V11J11-099__Z4_FO3_01__A1_small.npy,100


In [34]:
def get_numpy_datatype_unsigned_int(largest_value):
    if largest_value <= 255:
        print('In lower 255')
        value = np.uint8
    elif largest_value <= 65535:
        value = np.uint16
    else:
        value = 0
    return value

mymax = np.max(df_all['n_points']); print(mymax)
mydtype = get_numpy_datatype_unsigned_int(mymax); print(mydtype)
df_all['mydtype'] = mydtype
df_all

497
<class 'numpy.uint16'>


,Filenames,dims,radius,channel,n_points,method,Subfolder,Fpath_out,patchsize,mydtype
0,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,0,7,uniform,Ch_0_METHOD_uniform_npoints_7_radius_1/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>
1,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,1,7,uniform,Ch_1_METHOD_uniform_npoints_7_radius_1/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_1_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>
2,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,2,7,uniform,Ch_2_METHOD_uniform_npoints_7_radius_1/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_2_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>
3,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,0,13,uniform,Ch_0_METHOD_uniform_npoints_13_radius_2/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>
4,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,1,13,uniform,Ch_1_METHOD_uniform_npoints_13_radius_2/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_1_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>
5,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,2,13,uniform,Ch_2_METHOD_uniform_npoints_13_radius_2/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_2_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>
6,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,0,19,uniform,Ch_0_METHOD_uniform_npoints_19_radius_3/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_19_radius_3/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>
7,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,1,19,uniform,Ch_1_METHOD_uniform_npoints_19_radius_3/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_1_METHOD_uniform_npoints_19_radius_3/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>
8,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,2,19,uniform,Ch_2_METHOD_uniform_npoints_19_radius_3/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_2_METHOD_uniform_npoints_19_radius_3/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>
9,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",4,0,26,uniform,Ch_0_METHOD_uniform_npoints_26_radius_4/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_26_radius_4/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>


In [35]:
def subfolder_exists(directory_out_top, sub_directory_out):

    if not os.path.exists(directory_out_top+sub_directory_out):
        os.mkdir(directory_out_top+sub_directory_out)
        
    if os.path.exists(directory_out_top+sub_directory_out):
        return True
    else:
        return False
    
df_all['Subfolder_exists'] = [subfolder_exists(directory_out_top, subfolder) for
                             subfolder in df_all['Subfolder']]

print(np.all(df_all['Subfolder_exists']))
print(df_all['Subfolder_exists'].value_counts())
df_all

True
Subfolder_exists
True    45
Name: count, dtype: int64


,Filenames,dims,radius,channel,n_points,method,Subfolder,Fpath_out,patchsize,mydtype,Subfolder_exists
0,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,0,7,uniform,Ch_0_METHOD_uniform_npoints_7_radius_1/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True
1,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,1,7,uniform,Ch_1_METHOD_uniform_npoints_7_radius_1/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_1_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True
2,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,2,7,uniform,Ch_2_METHOD_uniform_npoints_7_radius_1/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_2_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True
3,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,0,13,uniform,Ch_0_METHOD_uniform_npoints_13_radius_2/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True
4,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,1,13,uniform,Ch_1_METHOD_uniform_npoints_13_radius_2/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_1_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True
5,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,2,13,uniform,Ch_2_METHOD_uniform_npoints_13_radius_2/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_2_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True
6,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,0,19,uniform,Ch_0_METHOD_uniform_npoints_19_radius_3/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_19_radius_3/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True
7,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,1,19,uniform,Ch_1_METHOD_uniform_npoints_19_radius_3/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_1_METHOD_uniform_npoints_19_radius_3/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True
8,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,2,19,uniform,Ch_2_METHOD_uniform_npoints_19_radius_3/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_2_METHOD_uniform_npoints_19_radius_3/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True
9,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",4,0,26,uniform,Ch_0_METHOD_uniform_npoints_26_radius_4/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_26_radius_4/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True


In [36]:
def fpath_out_exists(fpath_out):
        
    if os.path.exists(fpath_out):
        return True
    else:
        return False
    
df_all['Fpath_out_exists'] = [fpath_out_exists(fpath_out) for
                             fpath_out in df_all['Fpath_out']]

print(np.all(~df_all['Fpath_out_exists']))
print(df_all['Fpath_out_exists'].value_counts())
df_all

True
Fpath_out_exists
False    45
Name: count, dtype: int64


,Filenames,dims,radius,channel,n_points,method,Subfolder,Fpath_out,patchsize,mydtype,Subfolder_exists,Fpath_out_exists
0,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,0,7,uniform,Ch_0_METHOD_uniform_npoints_7_radius_1/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
1,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,1,7,uniform,Ch_1_METHOD_uniform_npoints_7_radius_1/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_1_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
2,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,2,7,uniform,Ch_2_METHOD_uniform_npoints_7_radius_1/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_2_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
3,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,0,13,uniform,Ch_0_METHOD_uniform_npoints_13_radius_2/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
4,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,1,13,uniform,Ch_1_METHOD_uniform_npoints_13_radius_2/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_1_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
5,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,2,13,uniform,Ch_2_METHOD_uniform_npoints_13_radius_2/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_2_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
6,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,0,19,uniform,Ch_0_METHOD_uniform_npoints_19_radius_3/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_19_radius_3/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
7,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,1,19,uniform,Ch_1_METHOD_uniform_npoints_19_radius_3/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_1_METHOD_uniform_npoints_19_radius_3/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
8,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,2,19,uniform,Ch_2_METHOD_uniform_npoints_19_radius_3/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_2_METHOD_uniform_npoints_19_radius_3/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
9,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",4,0,26,uniform,Ch_0_METHOD_uniform_npoints_26_radius_4/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_26_radius_4/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False


In [37]:
df_to_iter = df_all.loc[df_all['Fpath_out_exists'] == False]
df_to_iter

,Filenames,dims,radius,channel,n_points,method,Subfolder,Fpath_out,patchsize,mydtype,Subfolder_exists,Fpath_out_exists
0,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,0,7,uniform,Ch_0_METHOD_uniform_npoints_7_radius_1/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
1,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,1,7,uniform,Ch_1_METHOD_uniform_npoints_7_radius_1/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_1_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
2,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,2,7,uniform,Ch_2_METHOD_uniform_npoints_7_radius_1/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_2_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
3,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,0,13,uniform,Ch_0_METHOD_uniform_npoints_13_radius_2/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
4,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,1,13,uniform,Ch_1_METHOD_uniform_npoints_13_radius_2/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_1_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
5,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,2,13,uniform,Ch_2_METHOD_uniform_npoints_13_radius_2/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_2_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
6,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,0,19,uniform,Ch_0_METHOD_uniform_npoints_19_radius_3/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_19_radius_3/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
7,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,1,19,uniform,Ch_1_METHOD_uniform_npoints_19_radius_3/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_1_METHOD_uniform_npoints_19_radius_3/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
8,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,2,19,uniform,Ch_2_METHOD_uniform_npoints_19_radius_3/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_2_METHOD_uniform_npoints_19_radius_3/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False
9,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",4,0,26,uniform,Ch_0_METHOD_uniform_npoints_26_radius_4/,/home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-07e_LBP_parallel_values_no_crop_for_small_image/Ch_0_METHOD_uniform_npoints_26_radius_4/V11J11-099__Z4_FO3_01__A1_small.npy,100,<class 'numpy.uint16'>,True,False


In [38]:
list_of_images = list(pd.unique(df_to_iter['Filenames']))
print(len(list_of_images))
list_of_images

1


['V11J11-099__Z4_FO3_01__A1_small.tif']

In [39]:
@njit
def bincount_the_patches(lbp, patchsize, n_points):
    sizex0 = int(lbp.shape[0]/patchsize)
    sizex1 = int(lbp.shape[1]/patchsize)
    output = np.zeros((sizex0, sizex1, n_points+2), dtype=lbp.dtype)
    for i in range(sizex0):
        for j in range(sizex1):
            this_patch = lbp[i*patchsize:(i+1)*patchsize, 
                             j*patchsize:(j+1)*patchsize]
            mybincount = np.bincount(this_patch.ravel())
            output[i,j, 0:len(mybincount)] = mybincount
    return output


def apply_LBP_to_img(img, this_df, i):
    
    lbp = local_binary_pattern(img[:,:, this_df.iloc[i]['channel']], 
                               this_df.iloc[i]['n_points'], 
                               this_df.iloc[i]['radius'], 
                               this_df.iloc[i]['method']).astype(
        this_df.iloc[i]['mydtype'])
    
    lbp = uf.crop_to_superpixel(lbp, this_df.iloc[i]['patchsize'])
    lbp_bincounts = bincount_the_patches(lbp, 
                                         this_df.iloc[i]['patchsize'], 
                                         this_df.iloc[i]['n_points'])
    np.save(this_df.iloc[i]['Fpath_out'], lbp_bincounts)
#    print(lbp)
    return 0
#    np.save(this_df.iloc[i]['Fpath_out'], lbp)

def load_images(image_fname, df_all, directory_in_imgs):
    
#    img = np.load(directory_in_imgs + image_fname)
    img = skimage.io.imread(directory_in_imgs + image_fname)
    
    this_df = df_all.loc[df_all['Filenames'] == image_fname]
    
#    apply_LBP_to_img(img, this_df)

    Parallel(n_jobs=1)(delayed(apply_LBP_to_img)(img, this_df, 
                                                i) 
                   for i in 
                   trange(len(this_df)))
    
    
    
    return 0

In [40]:
#this_fname = df_all.iloc[0]['Filenames']
#this_outside_df = df_all.loc[df_all['Filenames'] == this_fname]
#img = np.load(directory_in_imgs + this_fname)
#lbp = apply_LBP_to_img(img, this_outside_df,
#                      0)
#display(this_outside_df)

In [41]:


#myindices = uf.get_block_indices_without_edges(lbp.shape[0], lbp.shape[1], 45)
#bincount_the_patches(lbp, 45, 7)


In [42]:
Parallel(n_jobs=16)(delayed(load_images)(img_fname, df_all, directory_in_imgs) 
                   for img_fname in 
                   tqdm(list_of_images,
                        total=len(list_of_images)))

  0%|          | 0/1 [00:00<?, ?it/s]

[0]

  0%|          | 0/45 [00:00<?, ?it/s]


## 